# Daily workflow - troselab (DRAFT)

`TR TODO: Jupyter ipywidget GUI`

(check Daniel's ipysheet mouse entry GUI for inspiration)

1. Upload to analysis server
    1. automatic (rcopy with file consolidation)?
    2. WinSCP?

    
2. Session ingest - Have this script parse the user folder. 
    1. retrieve login name  **DONE**
    2. parse user storage folder for SessionIDs, ScanIDs **DONE**
    3. intersect with 'Session' tables **DONE**
    4. FIRST STEP: session ingestion of new data  **DONE**
        1. have ipywidget with list of data to be ingested with checkbox  **DONE**
        2. have dropdowns next to SessionsFetch that expose 'Project', 'Anatomical Location', 'Equipment' (NOTE: have me or admin accept new entries? Or user?) - have a user default **DONE**
        3. have central 'START' button **DONE**
        5. OPTIONAL: Have a notebook section to enter and insert Session Notes
        6. OPTIONAL: Submit Sessions to pyrat. Not sure if necessary.
        7. TODO: Think if splitting projects off to different database suffixes (and differnt associated tables makes sense from here. Seems reasonable given the focus on the user root directory here.
        8. TODO: Implement auto animal pyrat ingest if needed. Crashes now.
        9. TODO: Make sure that multi-scan sessions are properly treated. Currently they are not detected as such by the GUI. Make sure ingestion behavior is appropriate.



3. Scan ingest / 1st stage s2p processing: 
    1. have another list with checkboxes (default checled) - now with all first stage scans of the newly ingested sessions. Have dropdown boxes with s2p parameteres. User default is selected. Followed by 'START'
    2. TODO: suppress s2p output, just have progress bars
    3. OPTIONAL: Have a notebook section to enter and insert Scan Notes
    4. OPTIONAL: Submit sessions, scans and Notes to RSPACE! Upload s2p average image, max image, Source image and Rastermap 
    

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.

5. SERVER: Database backup and Data folder backup
    1. Have cronjob backup database to ana2 and isilon (midnight, incremental - only add, no delete. Increment ID to be reversible?)
    2. Have cronjob backup data folders to ana2 and isilon (midnight, incremental - only add, no delete)

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [10]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery 
from adamacs.ingest import session as isess
import datajoint as dj




In [12]:
from rspace_client.eln import eln
import os


#API_KEY=os.environ.get('RSPACE_API_KEY')
#URL=os.environ.get('RSPACE_URL')
URL='https://rspace.rhrz.uni-bonn.de/'
API_KEY='SvMDsvzYNfJY36E0aHc5c2TriwsHWqs5'

api = eln.ELNClient(URL, API_KEY)

## sanity check that that the client is configured correctly
api.get_status()



#TR: check https://github.com/rspace-os/rspace-client-python/blob/9b2f0cffe82fd4e85b410daf6d574abcec5c1ca7/rspace_client/eln/eln.py#L798

{'message': 'OK', 'rspaceVersion': '1.80.1'}

In [13]:
folders = api.list_folder_tree()

In [14]:
for record in folders['records']:
    name = record['name']
    id = record['id']
    print(f"name: {name}, id: {id}")

name: Setups, id: 67423
name: Protocols, id: 67365
name: Experiments, id: 67345
name: Templates, id: 23795
name: Gallery, id: 23786
name: Examples, id: 23785
name: Shared, id: 23781


### parse through rspace folder strucutre to get animal ID folders

In [46]:
# find 'Experiments' folder and get ID
experiments_ids = []
for record in folders['records']:
    if record['name'] == 'Experiments':
        experiments_ids.append(record['id'])

names_and_ids = []
subfolders = api.list_folder_tree(experiments_ids[0])
names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])

In [47]:
# find matches with animal IDs
animalID = 'ROS-6666'

partial_matches = [x for sublist in names_and_ids for x in sublist if animalID in x[0]]
partial_matches_unique = set(partial_matches)

matching_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]

print(partial_matches)
print(partial_matches_unique)
print(matching_id)

partial_matches_dict = {'animalID': animalID, 'ID': [x[1] for x in partial_matches_unique][0]}


print(partial_matches_dict)




[('TR_ROS-6666', 67346)]
{('TR_ROS-6666', 67346)}
67346
{'animalID': 'ROS-6666', 'ID': 67346}


67346

### write an example document

In [49]:
# Set up folder ID and table HTML
table_html = '<table><tr><th>Header 1</th><th>Header 2</th></tr><tr><td>Data 1</td><td>Data 2</td></tr></table>'

# Create document metadata
doc_metadata = {
    'name': 'My Document',
    'tags': ['tag1', 'tag2'],
    'fields': {'field1': 'value1', 'field2': 'value2'},
    'folderId': matching_folder_id
}

# Create document with HTML content
doc_response = api.create_document(
    doc_metadata
#     content=table_html,
#     content_type='text/html'
)

# Print document ID
print(f"New document created with ID: {doc_response['id']}")


ApiError: Error code: 400, error message: JSON parse error: Cannot deserialize value of type `java.lang.String` from Object value (token `JsonToken.START_OBJECT`); nested exception is com.fasterxml.jackson.databind.exc.MismatchedInputException: Cannot deserialize value of type `java.lang.String` from Object value (token `JsonToken.START_OBJECT`)
 at [Source: (org.springframework.util.StreamUtils$NonClosingInputStream); line: 1, column: 10] (through reference chain: com.researchspace.api.v1.model.ApiDocument["name"]), errors: Either JSON syntax is invalid, or RSpace could not parse an expected date or numeric field

In [44]:
animal_ids

'TR_ROS-6666'

In [ ]:
db_prefix = dj.config['custom'].get('database.prefix')
print('database Prefix:', db_prefix)

database Prefix: tobiasr_


In [ ]:
eln

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment


Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## 1. Data upload
(not implemented currently)

## 2. Session ingest

### Define ingest GUI functions

In [5]:
# Tobias Rose 2023: Routine ingest helpers

import ipywidgets as widgets
import ipywidgets as widgets
from natsort import natsorted, ns
import re
from tqdm import tqdm

def select_sessions(SessionDirA, SessionDirB, Project, Recording_Location, Equipment, s2pparm):
    
    # Personal default values
    user_defaults = get_user_defaults(SessionDirB)
       
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for i, session_list in enumerate(SessionDirB):
        # Dropdowns for Project, Recording Location, Equipment
        project_dropdown = widgets.Dropdown(options=Project, value=Project[user_defaults[i][0]])
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=Recording_Location[user_defaults[i][1]])
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=Equipment[user_defaults[i][2]])
        s2pparms_dropdown = widgets.Dropdown(options=s2pparm, value=s2pparm[user_defaults[i][3]])
        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown, s2pparms_dropdown))
        # Checkbox for Process?
        session_checkbox = widgets.Checkbox(description='run?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append(session_checkbox)

    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(SessionDirB): #unique_directory_strings(SessionDirA, SessionDirB)
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='1800px')), 
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_dropdowns[i][3],
                session_checkboxes[i]
            ]
            if session_list in SessionDirB: #unique_directory_strings(SessionDirA, SessionDirB):
                hbox.children[0].value = '*' + session_list + ':'
                hbox.children[5].value = True
            else:
                hbox.children[5].value = False
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='0 0 auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [SessionDirB[i] for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_s2pparms = [session_dropdowns[i][3].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            
            selected_scans = get_scan_key_from_dir(selected_sessions)
            selected_sessions = get_session_key_from_dir(selected_sessions)
            
            
            output.clear_output()
            with output:
                print('Selected Sessions:', selected_sessions)
                # print('Selected Scans:', selected_scans)
                # print('Selected Projects:', selected_projects)
                # print('Selected Recording Locations:', selected_locations)
                # print('Selected Equipment:', selected_equipment)
                # print('Selected S2PParms:', selected_s2pparms)
            
                for sessi in selected_sessions:
                    isess.ingest_session_scan(sessi, verbose=True) 
                
                print('Ingested Sessions:', selected_sessions)

                # TODO: Make sure that sesssion ingest automatically pulls all data from pyrat!
                # TODO: Ingest Scans here
                # TODO: Queue Processing here

                # for idx, scansi in tqdm(selected_scans):
                    # isess.ingest_session_scan(sessi, verbose=True)

                # print('Ingested and process-queued Scans:', selected_scans)

        # Attach the callback function to the commit button
        commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)


def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

def unique_directory_strings(dirs1, dirs2):
    set1 = set(dirs1)
    set2 = set(dirs2)
    common_dirs = list(set1.intersection(set2))
    unique_dirs = list(set(set1.union(set2)) - set(common_dirs))
    return unique_dirs

def get_user_defaults(directory):
    RN = [1, 1, 2, 0]
    JJ = [2, 6, 2, 0]
    TR = [0, 6, 3, 0]
    DB = [4, 0, 1, 2]

    user_initials = get_user_initials_from_dir(directory)

    user_arrays = {}
    for initial in user_initials:
        values = locals()[initial]
        user_arrays[initial] = values

    new_array = [user_arrays[initial] for initial in user_initials]
    return new_array

#### Populate ingest GUI variables from DJ tables and manual arrays

In [7]:
# DJ tables
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')

dirs_ingested = session.SessionDirectory.fetch('session_dir')
SessionsDirArrayingested = get_session_dir_key_from_dir(dirs_ingested)
ScanDirArrayingested = get_scan_dir_key_from_dir(dirs_ingested)

# Manual arrays
s2pparm = ['mini2p_a', 'mini2p_b', 'bench2p_a']

#### Parse user data root dir and sort by date-string in directory name

In [8]:
dataroot = dj.config['custom']['exp_root_data_dir'][0]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and 'sess' in d]

sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)

### Generate GUI

In [9]:
print("\033[1m" + 'Select:                                           Project        Location  Setup        s2p parm' + "\033[0m" )
selected_data = select_sessions(SessionsDirArrayingested, sorted_dirs_root, Project, Recording_Location, Equipment, s2pparm)

Select:                                           Project        Location  Setup        s2p parm


Output()

In [17]:
session.Session()

session_id,subject PyRat import uses this for earmark value,session_datetime
sess9FB2LN5C,WEZ-8701,2022-03-18 00:00:00
sess9FF6TAW6,ROS-1438,2022-11-22 00:00:00
sess9FF6TL96,ROS-1438,2022-11-22 00:00:00
sess9FF6TT76,ROS-1438,2022-11-22 00:00:00
sess9FF6U114,ROS-1438,2022-11-22 00:00:00
sess9FG1SCD4,WEZ-8701,2023-01-13 00:00:00
sess9FG1TAXY,WEZ-8701,2023-01-13 00:00:00
sess9FG1UAQG,ROS-1438,2023-01-13 00:00:00
sess9FGLE1FN,OPI-1681,2023-02-15 00:00:00
sess9FGLEFJ3,OPI-1681,2023-02-15 00:00:00


### 2.1 Session notes ingest

## 3. Scan ingest / s2p processing 1st pass